#### Importing libraries

In [1]:
import numpy as np
import time
import cv2
import os
import glob
import matplotlib.pyplot as plt

In [2]:
def process_video(img):
    if img is not None:
        weights_path = os.path.join("Yolo", "yolov3.weights")
        config_path = os.path.join("Yolo", "yolov3.cfg")
        net = cv2.dnn.readNetFromDarknet(config_path , weights_path)
    
        (H, W) = (720, 1280)

        layers_names = ['yolo_82','yolo_94','yolo_106']
    
        blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416), crop=False, swapRB = False)
        net.setInput(blob)
        
        layers_output = net.forward(layers_names)
    
        boxes =[]
        confidences = []
        classIDs = []
    
        for output in layers_output:
            for detection in output:
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]
        
                if confidence >0.85:
                    box = detection[:4] * np.array([W , H, W, H])
                    bx, by, bw, bh = box.astype("int")
            
            
                    x = int(bx - (bw / 2))
                    y = int(by - (bh / 2))
            
            
                    boxes.append([x, y, int(bw), int(bh)])
                    confidences.append(confidence)
                    classIDs.append(classID)
    
        idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.8, 0.8)
    
        labels_path = os.path.join("Yolo", "coco.names.txt")
        labels = open(labels_path).read().strip().split("\n")
        
        if(len(idxs) > 0):
            for i in idxs.flatten():
                (x, y) = [boxes[i][0], boxes[i][1]]
                (w, h) = [boxes[i][2], boxes[i][3]]
    
                cv2.rectangle(img, (x, y), (x + w, y + h), (50,50,250), 2)
                text = f"{labels[classIDs[i]]}: {confidences[i]:.3f}"
                # calculate text width & height to draw the transparent boxes as background of the text
                (text_width, text_height) = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, thickness=2)[0]
                text_offset_x = x
                text_offset_y = y - 5
                box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height))
                overlay = img.copy()
                cv2.rectangle(overlay, box_coords[0], box_coords[1], color=(50,50,250), thickness=cv2.FILLED)
                # add opacity (transparency to the box)
                img = cv2.addWeighted(overlay, 0.6, img, 0.4, 0)
                # now put the text (label: confidence %)
                cv2.putText(img, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale = 1, color=(0, 0, 0), thickness=2)
        
        return img

#### Import everything needed to edit/save/watch video clips

In [3]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#### Choose your processing video

In [4]:
clip2_path = os.path.join("test_videos", "challenge_video.mp4")
clip2 = VideoFileClip(clip2_path)
challenge_output = 'challenge_video_output.mp4'
challenge_clip = clip2.fl_image(process_video) #NOTE: this function expects color images!!
%time challenge_clip.write_videofile(challenge_output, audio=False) 

Moviepy - Building video challenge_video_output.mp4.
Moviepy - Writing video challenge_video_output.mp4



Moviepy - Done !
Moviepy - video ready challenge_video_output.mp4
CPU times: total: 28min 1s
Wall time: 8min 44s


In [8]:
videoClip = VideoFileClip("challenge_video_output.mp4")
videoClip.write_gif("demo.gif")

MoviePy - Building file demo.gif with imageio.
